## Module 4 Lab - Anomaly Detection

_Anomaly detection (also outlier detection) is the identification of items, events or observations which do not conform to an expected pattern or other items in a dataset - wikipedia.com_

Most of the times outliers are a by product of clustering algorithms. Clustering algorithms are not designed to detect outliers but rather to form clusters of data. The accuracy of how well outliers are detected depends on how well clustering algorithms can capture the structures. Sometimes a set of similar abnormal objects are group as a cluster. 

#### Outliers

Outliers are the samples that are exceptionally far from the mainstream of data. There is no rigid mathematical definition of what constitutes an outlier; determining whether or not an observation is an outlier is ultimately a subjective exercise. There are various methods of outlier detection. Some are graphical such as normal probability plots. Others are model-based.

There are two main types of outliers, representative and nonrepresentative. A representative outlier is one that is a correct or valid observation that "cannot be regarded as unique". While this type of outlier is considered an extreme value, it should be retained, with special treatment during the analysis stages. A nonrepresentative outlier is one that is an "incorrect observation" (i.e., due to an error in data entry, coding, or measurement) or is considered unique because there are no other values like it in the population. Nonrepresentative outliers should be corrected or excluded from the analysis.

Predictive modeling techniques can be impacted as a result of presence of outliers. Dealing with outliers in data analysis is one of the important challenges. Detecting outliers and understanding them can lead to interesting findings. Outliers can impact accuracy of predictive models. Detecting outliers and dealing with them is a critical step in data preparation for predictive modeling. Below picture shows how an outlier can impact the overall fit of a linear regression model. Lets discuss how PCA technique can be used for detecting outliers in multivariate datasets. 

<img src="../Images/outlier.PNG">

#### Methods to detect outliers:

There are several approaches for detecting Outliers. Charu Aggarwal in his book [Outlier Analysis](http://www.charuaggarwal.net/outlierbook.pdf) classifies Outlier detection models in following groups:

**Extreme Value Analysis:** This is the most basic form of outlier detection and only good for 1-dimension data. In these types of analysis, it is assumed that values which are too large or too small are outliers. Z-test and Student’s t-test are examples of these statistical methods. These are good heuristics for initial analysis of data but they don’t have much value in multivariate settings. They can be used as final steps for interpreting outputs of other outlier detection methods.

**Probabilistic and Statistical Models:** These models assume specific distributions for data. Then using the expectation-maximization(EM) methods they estimate the parameters of the model. Finally, they calculate probability of membership of each data point to calculated distribution. The points with low probability of membership are marked as outliers.

**Linear Models:** These methods model the data into a lower dimensional sub-spaces with the use of linear correlations. Then the distance of each data point to plane that fits the sub-space is being calculated. This distance is used to find outliers. PCA(Principal Component Analysis) is an example of linear models for anomaly detection.

**Proximity-based Models:** The idea with these methods is to model outliers as points which are isolated from rest of observations. Cluster analysis, density based analysis and nearest neighborhood are main approaches of this kind.

**Information Theoretic Models:** The idea of these methods is the fact that outliers increase the minimum code length to describe a data set. High-Dimensional Outlier Detection: Specifc methods to handle high dimensional sparse data

#### Outlier Detection Using Principal Component Analysis

PCA is a statistical procedure that uses a transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components. The number of principal components is less than or equal to the number of original variables. This transformation is defined in such a way that the first principal component has the largest possible variance (that is, accounts for as much of the variability in the data as possible), and each succeeding component in turn has the highest variance possible under the constraint that it is orthogonal to the preceding components. The resulting vectors are an uncorrelated orthogonal basis set.

Using PCA we can map our dataset with n-dimension (possibly correlated variables) to a k-dimensional sub-space of k uncorrelated components (k<=n).

Below are steps for detecting anomalies using PCA:

First we map the data set from its original n-dimensional space to k-dimensional subspace using PCA

- Calculate the centroid of data points (μ)
- Calculate the variance of each component (λ)
- Calculate the score of each data point using below formula:

<img src="../Images/PCA_formula.png">

- Finally, use extreme value analysis methods to find data points with extreme scores

You dont have to worry about applying above formula to calculate scores. PCA is alreday implemented. You just have to call the method to find scores.

In [ ]:
# install.packages("psych",dependencies = TRUE, repos = c(CRAN="https://cran.r-project.org/"))
library(psych)

In [ ]:
# First we load the diamonds dataset
library(ggplot2)
data(diamonds)

In [ ]:
head(diamonds)

In [ ]:
# Run the cell to know more about data
help(diamonds)

In [ ]:
# Convert the categorical variables to dummy variables as PCA doesn't run with character type factors. dummy variables 
# are explained below with example cde.
# Use dummy.code from psych package
codedData <- cbind(diamonds,dummy.code(diamonds$cut),dummy.code(diamonds$color),dummy.code(diamonds$clarity))

**Reference: ** [Dummy coding the data](http://stats.idre.ucla.edu/r/library/r-library-contrast-coding-systems-for-categorical-variables/)

Below piece of code illustrates how dummy coding words

In [ ]:
# Generate example dataframe with character column
example <- as.data.frame(c("A", "A", "B", "F", "C", "G", "C", "D", "E", "F"))
names(example) <- "strcol"

# For every unique value in the string column, create a new 1/0 column
# This is what Factors do "under-the-hood" automatically when passed to function requiring numeric data
for(level in unique(example$strcol)){
  example[paste("dummy", level, sep = "_")] <- ifelse(example$strcol == level, 1, 0)
}
example

From output of below str() command, you can see five new columns 11 to 15 are created for the corresponding 5 levels in cut variable. Like wise, seven new columns are created for 7 levels in color factor variable and finally eight new dummy columns are created for the 8 levels in clarity variable. 

In [ ]:
str(codedData)

Columns 2, 3, 4 are factors with charaters values. Remove these columns from data as PCA can only work with numeric data. 

In [ ]:
codedData <- codedData[,-c(2,3,4)]

Use the prcomp method to find Principal components

In [ ]:
pr <- prcomp(codedData,center = TRUE, scale. = TRUE)

**Scores: ** The positions of each observation in this new coordinate system of principal components are called scores and are calculated as linear combinations of the original variables and the weights aij. For example, the score for the rth sample on the kth principal component is calculated as

$$Y_{kr} = a_{k1}x_{k1} + a_{k2}x_{k2} + ... + a_{kp}x_{kp}$$

**Reference: ** [PCA tutorial](http://strata.uga.edu/software/pdf/pcaTutorial.pdf)

These scores are nothing but the principal components. 

We want to sum all the principal component values for each row which will be score for the row.

In [ ]:
components <- pr$x
scores <- rowSums(components)
scores[1:10]

In [ ]:
?boxplot

Plot the histogram and boxplot for the scores calculated

In [ ]:
options(scipen=999)
par(mfrow=c(2,1));

#Create histogram of scores
hist(scores,breaks = 100)

#Drawing Boxplot
boxplot(scores,horizontal = FALSE)

Most of the rows have scores in the range of 100 to 500. Check rows with scores > 5000

In [ ]:
print(range(scores))

In [ ]:
length(which(scores>25))
diamonds[which(scores>25),]

Look at the y value in first row. It clearly suggests y value is incorrect. It looks like the depth value has been incorrectly used for y. 

## Local Outlier Factor

Local outlier factor is another way of finding outliers. Read the below wiki page to learn how this method works. 

**Reference: ** 
- [Wiki](https://en.wikipedia.org/wiki/Local_outlier_factor)

### The LOF algorithm

**Reference: ** [lofactor()](https://www.rdocumentation.org/packages/DMwR/versions/0.4.1/topics/lofactor)

In [ ]:
library(DMwR)

In [ ]:
# # point to the prostate data set in the h2o folder - no need to load h2o in memory yet

prostate_df <- read.table("/dsa/data/DSA-8630/prostate.txt",sep=',',header=TRUE)
head(prostate_df)

In [ ]:
dim(prostate_df)

In [ ]:
# # We don't need the ID field
prostate_df <- prostate_df[,-1]
summary(prostate_df)

Using lofactor(), you are just telling the function how many neighbours it should consider by giving a number to k when calculating the outliers. 

In [ ]:
# k=5 indicates to pick top 5 outliers
outlier.scores <- lofactor(prostate_df, k=50)

Now that the outlier scores are calculated, pick the top 10 outliers with highest scores.

In [ ]:
outliers <- order(outlier.scores, decreasing=T)[1:10]

# who are outliers
print(outliers)

#### Visualize Outliers with Plots

Next, we show outliers with a biplot of the first two principal components.

In [ ]:
n <- nrow(prostate_df)
labels <- 1:n
labels[-outliers] <- "."
biplot(prcomp(prostate_df), cex=.8, xlabs=labels)

Its up to you to decide if below values are outliers and removed from dataset. It requires domain knowledge or knowledge about the data to make decisions on data. You may want to do more analysis before excluding them. 

In [ ]:
prostate_df[outliers,]

### Outliers Detection in Time Series data

In [ ]:
library(tsoutliers)
library(expsmooth)
library(fma)

The data in table_a has the historical stock data of different companies. It has following fields Date, Time, Open, High, Low, Close, Volume.  


- Date – This provides the date as an integer where 20100527 would represent May 27th, 2010.
- Time – This gives the time as an integer where 1426 would represent 2:26PM EST.
- Open – The open price.
- High – The high price.
- Low – The low price.
- Close – The close price.
- Volume – The trading volume during the interval. Note that it is extremely difficult to get accurate volume information. The volume is adjusted for splits so that the total value of shares traded remains constant even if a split occurs.

In [ ]:
names=c("Date","Time","Open","High","Low","Close","Volume")
stock = read.csv("/dsa/data/DSA-8630/table_a.csv",header=FALSE)
names(stock) = names

In [ ]:
head(stock)

Convert the Close column into a time series data. start and end has the time periods for which the data is available. Since we have the data for everyday (although there are some missing dates) we will use frequency=30.

In [ ]:
stock_series <- ts(stock$Close, start=c(1999, 11, 18), end=c(2013, 08, 09), frequency=30)

In [ ]:
stock_series[1:100]

The package detects 5 different types of outliers iteratively in time series data:

- Additive Outlier (AO)
- Innovation Outlier (IO)
- Level Shift (LS)
- Temporary change (TC)
- Seasonal Level Shift (SLS)

[Click here for definition of these outliers](https://www.ibm.com/support/knowledgecenter/SS3RA7_15.0.0/com.ibm.spss.modeler.help/ts_outliers_overview.htm)

In below cell we are looking for Additive, Level shift and temporarry chnage outliers in data.

In [ ]:
stock_outliers <- tso(stock_series,types = c("AO","LS","TC"),maxit.iloop=10)
stock_outliers
plot(stock_outliers)

## Extra

-----


**Have you used apply() before.. Here's something similar but does little more than that** 

- [sweep()](https://stat.ethz.ch/R-manual/R-devel/library/base/html/sweep.html)

**Syntax:** sweep(x, MARGIN, STATS, FUN="-", check.margin=T, ...)


sweep is similar to apply() where yu apply a function to each column or row of a dataframe. But sweep is typically used when you operate a matrix by row or by column, and need the flexibility for the input of the operation to have a different value for each row / column. Whether you operate by row(1) or column(2) is defined by MARGIN, the second parameter in below code. So in below piece of code, for each column you will take a value from c(10, 20, 30) which is being defined by STATS parameter and use in the operation "+" defined by FUN parameter.

In [ ]:
# Sample code explaining what sweep() is doing
a = c(130,110,118,112,128)
b = c(26,24,25,25,26)
c = c(140,155,142,175,170)
names=c("Weight","Waist","Height")
size = data.frame(a,b,c)
names(size)=names
print(size)

# We are adding values 10, 20, 30 to columns 
sweep(size, 2, c(10, 20, 30), "+")
